In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('sqlite_db_pythonsqlite.db')

In [3]:
curs = conn.cursor()

In [4]:
curs.execute("SELECT name FROM sqlite_master WHERE type = 'table'").fetchall()

[('Bookings',), ('Facilities',), ('Members',)]

In [5]:
curs.execute('SELECT * FROM Bookings').description

(('bookid', None, None, None, None, None, None),
 ('facid', None, None, None, None, None, None),
 ('memid', None, None, None, None, None, None),
 ('starttime', None, None, None, None, None, None),
 ('slots', None, None, None, None, None, None))

In [6]:
curs.execute('SELECT * FROM Facilities').description

(('facid', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('membercost', None, None, None, None, None, None),
 ('guestcost', None, None, None, None, None, None),
 ('initialoutlay', None, None, None, None, None, None),
 ('monthlymaintenance', None, None, None, None, None, None))

In [7]:
curs.execute('SELECT * FROM Members').description


(('memid', None, None, None, None, None, None),
 ('surname', None, None, None, None, None, None),
 ('firstname', None, None, None, None, None, None),
 ('address', None, None, None, None, None, None),
 ('zipcode', None, None, None, None, None, None),
 ('telephone', None, None, None, None, None, None),
 ('recommendedby', None, None, None, None, None, None),
 ('joindate', None, None, None, None, None, None))

Now we have all three tables, and the columns of each table identified

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! 

In [8]:
q10 = curs.execute('SELECT name, revenue FROM(SELECT name, SUM(CASE WHEN memid = 0 THEN guestcost * slots ELSE membercost * slots END) AS revenue FROM Bookings INNER JOIN Facilities ON Bookings.facid = Facilities.facid GROUP BY name) AS inner_table WHERE revenue < 1000 ORDER BY revenue')

In [9]:
q10 = curs.fetchall()
print(q10)

[('Table Tennis', 180), ('Snooker Table', 240), ('Pool Table', 270)]


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [10]:
list11 = curs.execute('SELECT surname || ", " || firstname AS name, recommendedby FROM Members WHERE recommendedby > 0 ORDER BY name')

In [11]:
list11 = curs.fetchall()

In [12]:
print(list11)

[('Bader, Florence', '9'), ('Baker, Anne', '9'), ('Baker, Timothy', '13'), ('Boothe, Tim', '3'), ('Butters, Gerald', '1'), ('Coplin, Joan', '16'), ('Crumpet, Erica', '2'), ('Dare, Nancy', '4'), ('Genting, Matthew', '5'), ('Hunt, John', '30'), ('Jones, David', '4'), ('Jones, Douglas', '11'), ('Joplette, Janice', '1'), ('Mackenzie, Anna', '1'), ('Owen, Charles', '1'), ('Pinker, David', '13'), ('Purview, Millicent', '2'), ('Rumney, Henrietta', '20'), ('Sarwin, Ramnaresh', '15'), ('Smith, Jack', '1'), ('Stibbons, Ponder', '6'), ('Worthington-Smyth, Henry', '2')]


Q12: Find the facilities with their usage by member, but not guests

In [13]:
q12 = curs.execute('SELECT f.name, firstname|| " "|| surname as member_name, sum(slots), COUNT(f.name) FROM Facilities as f  INNER JOIN Bookings as b  ON f.facid = b.facid  INNER JOIN Members as m  ON b.memid = m.memid  WHERE m.memid > 0 GROUP BY f.name, member_name')

In [14]:
q12 = curs.fetchall()
print(q12)

[('Badminton Court', 'Anna Mackenzie', 96, 30), ('Badminton Court', 'Anne Baker', 30, 10), ('Badminton Court', 'Burton Tracy', 6, 2), ('Badminton Court', 'Charles Owen', 18, 6), ('Badminton Court', 'Darren Smith', 432, 132), ('Badminton Court', 'David Jones', 24, 8), ('Badminton Court', 'David Pinker', 21, 7), ('Badminton Court', 'Douglas Jones', 6, 2), ('Badminton Court', 'Erica Crumpet', 6, 2), ('Badminton Court', 'Florence Bader', 27, 9), ('Badminton Court', 'Gerald Butters', 63, 20), ('Badminton Court', 'Henry Worthington-Smyth', 15, 4), ('Badminton Court', 'Hyacinth Tupperware', 3, 1), ('Badminton Court', 'Jack Smith', 36, 12), ('Badminton Court', 'Jemima Farrell', 21, 7), ('Badminton Court', 'John Hunt', 6, 2), ('Badminton Court', 'Millicent Purview', 6, 2), ('Badminton Court', 'Nancy Dare', 30, 10), ('Badminton Court', 'Ponder Stibbons', 48, 16), ('Badminton Court', 'Ramnaresh Sarwin', 21, 7), ('Badminton Court', 'Tim Boothe', 36, 12), ('Badminton Court', 'Tim Rownam', 12, 4), (

In [15]:
q12df = pd.DataFrame(q12)

In [16]:
q12df

,0,1,2,3
0,Badminton Court,Anna Mackenzie,96,30
1,Badminton Court,Anne Baker,30,10
2,Badminton Court,Burton Tracy,6,2
3,Badminton Court,Charles Owen,18,6
4,Badminton Court,Darren Smith,432,132
...,...,...,...,...
197,Tennis Court 2,Ramnaresh Sarwin,36,11
198,Tennis Court 2,Tim Boothe,168,52
199,Tennis Court 2,Tim Rownam,18,6
200,Tennis Court 2,Timothy Baker,21,7


Q13: Find the facilities usage by month, but not guests

In [17]:
q13 = curs.execute('SELECT f.name, firstname|| " "|| surname as member_name, strftime("%m", starttime), sum(slots), COUNT(f.name) FROM Facilities as f  INNER JOIN Bookings as b  ON f.facid = b.facid  INNER JOIN Members as m  ON b.memid = m.memid  WHERE m.memid > 0 GROUP BY f.name, member_name, strftime("%m", starttime)')

In [18]:
q13 = curs.fetchall()
print(q13)

[('Badminton Court', 'Anna Mackenzie', '08', 9, 3), ('Badminton Court', 'Anna Mackenzie', '09', 87, 27), ('Badminton Court', 'Anne Baker', '08', 3, 1), ('Badminton Court', 'Anne Baker', '09', 27, 9), ('Badminton Court', 'Burton Tracy', '08', 3, 1), ('Badminton Court', 'Burton Tracy', '09', 3, 1), ('Badminton Court', 'Charles Owen', '08', 12, 4), ('Badminton Court', 'Charles Owen', '09', 6, 2), ('Badminton Court', 'Darren Smith', '07', 123, 38), ('Badminton Court', 'Darren Smith', '08', 180, 55), ('Badminton Court', 'Darren Smith', '09', 129, 39), ('Badminton Court', 'David Jones', '08', 12, 4), ('Badminton Court', 'David Jones', '09', 12, 4), ('Badminton Court', 'David Pinker', '08', 6, 2), ('Badminton Court', 'David Pinker', '09', 15, 5), ('Badminton Court', 'Douglas Jones', '09', 6, 2), ('Badminton Court', 'Erica Crumpet', '09', 6, 2), ('Badminton Court', 'Florence Bader', '08', 15, 5), ('Badminton Court', 'Florence Bader', '09', 12, 4), ('Badminton Court', 'Gerald Butters', '07', 12